In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11038202501300727220
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12734127457086902983
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8640022883109541903
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11276946637
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6137422874850764075
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [0]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.applications.resnet50 import ResNet50
from keras import callbacks
import numpy as np

Using TensorFlow backend.


In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:

"""
Parameters
"""
epochs = 100
img_width, img_height =300,300
classes_num = 5
batch_size=32
arsitektur = 'ResNet50'
#lr = 0.0004

In [0]:
DATA_DIR = '/content/gdrive/My Drive/CITS Color'

train_datagen = ImageDataGenerator(rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    DATA_DIR, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data


Found 1079 images belonging to 5 classes.
Found 268 images belonging to 5 classes.


In [0]:


model = ResNet50(input_shape=(300,300,3),include_top=True, classes=5, pooling='max',weights=None)

model.summary()


architecure_dir = os.path.join('/content/gdrive/My Drive/Colab Notebooks/LeafClassification/',arsitektur)
if not os.path.exists(architecure_dir):
  os.mkdir(architecure_dir)
# Save the model architecture
with open(os.path.join(architecure_dir,'architecture.json'), 'w') as f:
    f.write(model.to_json())



Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 150, 150, 64) 256         conv1[0][0]                      
_____________________________________

In [0]:
from keras.models import model_from_json
with open(os.path.join(architecure_dir,'architecture.json'), 'r') as f:
    model = model_from_json(f.read())

In [0]:
# (4) Compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
"""
Tensorboard log
"""


log_dir = os.path.join(architecure_dir, 'tf-log')
tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=True)
cbks = [tb_cb]

"""model.fit_generator(
    train_generator,
    samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=cbks,
    validation_steps=validation_steps)"""

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    callbacks=cbks,
    epochs = epochs,
    shuffle = True)

#model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), steps_per_epoch=len(x_train) / 32, epochs=epochs)

target_dir =os.path.join(architecure_dir, 'models')
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
  
model.save(os.path.join(target_dir, 'model.h5'))
model.save_weights(os.path.join(target_dir, 'weights.h5'))

Epoch 1/100
33/33 [==============================] - 56s 2s/step - loss: 0.1198 - acc: 0.9532 - val_loss: 0.3578 - val_acc: 0.8729
Epoch 2/100
33/33 [==============================] - 53s 2s/step - loss: 0.1548 - acc: 0.9434 - val_loss: 1.3299 - val_acc: 0.6780
Epoch 3/100
33/33 [==============================] - 53s 2s/step - loss: 0.1194 - acc: 0.9608 - val_loss: 1.4106 - val_acc: 0.6568
Epoch 4/100
33/33 [==============================] - 53s 2s/step - loss: 0.0944 - acc: 0.9627 - val_loss: 0.2373 - val_acc: 0.9195
Epoch 5/100
33/33 [==============================] - 53s 2s/step - loss: 0.0941 - acc: 0.9661 - val_loss: 2.6286 - val_acc: 0.5720
Epoch 6/100
33/33 [==============================] - 54s 2s/step - loss: 0.1102 - acc: 0.9621 - val_loss: 1.1037 - val_acc: 0.6562
Epoch 7/100
33/33 [==============================] - 53s 2s/step - loss: 0.1069 - acc: 0.9542 - val_loss: 1.7187 - val_acc: 0.5847
Epoch 8/100
33/33 [==============================] - 53s 2s/step - loss: 0.0965 - a